In [ ]:
# importing libraries
from collections import namedtuple

import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [76]:
sc = SparkContext()
# we initiate the StreamingContext with 10 second batch interval. 
# next we initiate our sqlcontext

ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint")
sqlContext = SQLContext(sc)

In [77]:
host =  "192.168.56.1"
port = 4041
socket_stream = ssc.socketTextStream(host, port)
lines = socket_stream.window(60)

In [80]:
words = lines.flatMap(lambda text: text.split(" ")).filter(lambda text: text.lower().startswith('#'))

Hashtag = namedtuple('Hashtag', ("tag", "count"))

words.map(lambda word: (word.lower(), 1))\
    .reduceByKey( lambda a, b: a + b )\
    .map(lambda rec: Hashtag(rec[0], rec[1]))\
    .foreachRDD(lambda rdd: rdd.toDF()\
    .sort(desc("count")).limit(10)\
    .registerTempTable("hashtags"))


Tweet = namedtuple('Tweet', ('text'))
lines.map(lambda p: Tweet(p[0])).foreachRDD(lambda rdd: rdd.toDF().registerTempTable("tweets"))

In [81]:
ssc.start()

In [82]:

time.sleep( 60 )
count = 0
top_10_from_all_windows = None
tweets_df = None
while count < 1:
    time.sleep( 5 )
    top_10_tags = sqlContext.sql('Select tag, count from hashtags')
    print(top_10_tags)
    top_10_df = top_10_tags.toPandas()
    
    if top_10_from_all_windows is None:
        top_10_from_all_windows = top_10_df
    else:
        top_10_from_all_windows = pd.concat([top_10_from_all_windows, top_10_df])
      
    
    tweets_window = sqlContext.sql('Select text from tweets')
    print(tweets_window)
    tweets_window_df = tweets_window.toPandas()
    if tweets_df is None:
        tweets_df = tweets_window_df
    else:
        tweets_df = pd.concat([tweets_df, tweets_window_df])
    
    count += 1

ssc.stop()

DataFrame[tag: string, count: bigint]


AnalysisException: 'Table or view not found: tweets; line 1 pos 17'

In [74]:
ssc.stop()

In [ ]:

plt.figure(figsize = (10, 8))
sns.barplot(x="count", y="hashtag", data=top_10_df)
plt.show()
count = count + 1
    

In [26]:
grouped = top_10_from_all_windows.groupby('tag').sum().reset_index()
grouped.sort_values('count', ascending=False)

,tag,count
1,#bts,223
11,#방탄소년단,204
6,#gateway,44
12,#아스트로,31
7,#knock,29
0,#astro,25
10,#twi…rt,14
5,#controlonspotify,11
8,"#rainonme,",8
4,#control,6


In [ ]:
tweets_df